# Football Player Value
- Data From : https://www.transfermarkt.com/
    * 트랜스퍼마켓 : 공신력 있는 축구선수 이적 관련 사이트
- Subject : 축구선수 몸값은 어떻게 결정될까?
    * 1. 축구가 산업화된 잉글랜드 프리미어리그에선 영국 국적이 몸값에 영향을 준다고 한다. 정말?
    * 2. 골, 어시스트, 키, 몸무게, 국적, 소속팀, 소속팀 성적, 소속팀 소속 리그 등 다양한 변수 활용
- Project Process
    * 1. 웹크롤링으로 데이터 확보
    * 2. 확보한 데이터 병합
    * 3. EDA 및 데이터전처리
    * 4. 서브주제에 맞춰 다양한 ML모델 사용
- Stack : Beautiful Soup 4, Pandas, Seaborn, Scikit-Learning

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import re
# 정규표현식

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV # 데이터 나누기, 점검 및 훈련
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_score, recall_score # 평가지표
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline
import multiprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## 1. 축구선수 이적 데이터 확보
- webcrawling
- transfer markt